<a href="https://colab.research.google.com/github/pintophilip/Agentic-AI-Application/blob/main/langgraph_maths_llama_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangGraph Agent Using LLM and Custom Math Functions
This notebook demonstrates how to use LangGraph and a Groq-compatible LLM to answer both general and mathematical questions using custom functions.

In [1]:
# 🛠️ Install Required Libraries
!pip install --upgrade langgraph langchain langchain_groq

# 🔐 Set up your Groq API Key
import os
os.environ["GROQ_API_KEY"] = "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"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.50:
      Successfully uninstalled langchain-core-0.3.50
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-s

In [55]:
# 🔣 Import Libraries
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
import re

In [4]:
# Define Custom Math Tools

@tool(description="Adds two numbers together.")
def plus(a: float, b: float) -> float:
    return a + b

@tool(description="Subtracts two numbers.")
def subtract(a: float, b: float) -> float:
    return a - b

@tool(description="Multiplies two numbers.")
def multiply(a: float, b: float) -> float:
    return a * b

@tool(description="Divides two numbers.")
def divide(a: float, b: float) -> float:
    try:
        return a / b
    except ZeroDivisionError:
        return "Cannot divide by zero."


In [54]:
# Set Up Mistral LLM via Groq
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=os.getenv("GROQ_API_KEY")
)

# Call LLM correctly
response = llm.invoke([HumanMessage(content="Who is the president of India?")])
print(response.content)

The President of India is Droupadi Murmu. She took office on July 25, 2022, and is the 15th President of India.


In [45]:
# 🧾 Define Agent State
class AgentState(TypedDict):
    query: str
    result: Optional[str]

In [46]:
# 🔍 Decide if query is math or general
def detect_query_type(state: AgentState) -> str:
    query = state["query"].lower()
    if any(word in query for word in ["plus", "add", "minus", "subtract", "multiply", "times", "divide"]):
        return "math_node"
    return "llm_node"

In [56]:
def math_node(state: AgentState) -> AgentState:
    query = state["query"].lower()

    # Extract numbers using regex (robust for negatives, decimals, punctuation)
    nums = [float(n) for n in re.findall(r"-?\d+(?:\.\d+)?", query)]

    if len(nums) < 2:
        return {"query": state["query"], "result": "Sorry, I need two numbers for this operation."}

    if "plus" in query or "add" in query or "sum" in query:
        result = plus.invoke({"a": nums[0], "b": nums[1]})
    elif "minus" in query or "subtract" in query or "difference" in query:
        result = subtract.invoke({"a": nums[0], "b": nums[1]})
    elif "multiply" in query or "times" in query or "product" in query:
        result = multiply.invoke({"a": nums[0], "b": nums[1]})
    elif "divide" in query or "divided" in query:
        result = divide.invoke({"a": nums[0], "b": nums[1]})
    else:
        return {"query": state["query"], "result": "Sorry, I didn't understand the math operation."}

    return {"query": state["query"], "result": str(result)}

In [57]:
# 💬 Handle General LLM Queries
def llm_node(state: AgentState) -> AgentState:
    response = llm.invoke([HumanMessage(content=state["query"])])
    return {"query": state["query"], "result": response.content}

In [58]:
# ✅ Build the LangGraph
builder = StateGraph(AgentState)
builder.add_node("math_node", math_node)
builder.add_node("llm_node", llm_node)
builder.set_conditional_entry_point(detect_query_type)
builder.add_edge("math_node", END)
builder.add_edge("llm_node", END)
graph = builder.compile()

# ✅ Test the agent
print("➡️ Math Test:")
output = graph.invoke({"query": "What is 7 plus 5?"})
print("Response:", output["result"])

print("\n➡️ General Knowledge Test:")
output = graph.invoke({"query": "Who is the president of India?"})
print("Response:", output["result"])


➡️ Math Test:
Response: 12.0

➡️ General Knowledge Test:
Response: The President of India is Droupadi Murmu. She took office on July 25, 2022, and is the 15th President of India. Prior to her presidency, she served as the Governor of Jharkhand from 2015 to 2021.


In [61]:
def test_math_query():
    """Tests the math_node function with various math queries."""

    # Define test cases
    test_cases = [
        ("What is 6 plus 4?", "10.0"),
        ("What is 10 minus 5?", "5.0"),
        ("What is 3 times 7?", "21.0"),
        ("What is 20 divided by 4?", "5.0"),
        ("What is 5 plus -2?", "3.0"),  # Test with negative numbers
        ("Calculate 15 minus 7", "8.0"),  # Test without question mark
        ("What is 6 plus?", "Sorry, I need two numbers for this operation."), # Test with missing number

    ]

    # Run tests and assert results
    for query, expected_result in test_cases:
        output = graph.invoke({"query": query})
        assert output["result"] == expected_result, f"Test failed for query: '{query}'. Expected: '{expected_result}', Got: '{output['result']}'."

    print("All math query tests passed!")

# Call the test function
test_math_query()

All math query tests passed!
